## Import libraries

In [16]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.pyplot import axes, title
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
import seaborn as sns

## Load the data

In [17]:
# Load the data set for the fourth IoT node
benign_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.benign.csv")
#g_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.scan.csv")
#g_junk_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.junk.csv")
#g_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.udp.csv")
#g_tcp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.tcp.csv")
g_combo_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.combo.csv")
#m_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.scan.csv")
#m_ack_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.ack.csv")
#m_syn_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.syn.csv")
#m_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udp.csv")
#m_p_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udpplain.csv")

In [18]:
#print
#print('Shape==>', 'Benign shape: ', benign_4.shape, '//','Bashlite scan: ', g_scan_4.shape, '//','Bashlite junk: ', g_junk_4.shape, '//','Bashlite udp: ', g_udp_4.shape,
#      '//','Bashlite tcp: ', g_tcp_4.shape, '//','Bashlite combo: ', g_combo_4.shape, '//','Mirai scan: ', m_scan_4.shape, '//','Mirai ack: ', m_ack_4.shape, '//','Mirai syn: ', m_syn_4.shape, '//','Mirai udp: ', m_udp_4.shape, '//','Mirai udpplain: ', m_p_udp_4.shape)

In [19]:
print
print('Shape==>', 'Benign shape: ', benign_4.shape, '//','Bashlite combo: ', g_combo_4.shape)

Shape==> Benign shape:  (175240, 115) // Bashlite combo:  (58152, 115)


## Prepare the data

In [20]:
benign=benign_4.sample(frac=0.25,replace=False)
g_combo=g_combo_4.sample(frac=0.25,replace=False)
#g_junk=g_junk_4.sample(frac=0.5,replace=False)
#g_scan=g_scan_4.sample(frac=0.5,replace=False)
#g_tcp=g_tcp_4.sample(frac=0.15,replace=False)
#g_udp=g_udp_4.sample(frac=0.15,replace=False)
#m_ack=m_ack_4.sample(frac=0.25,replace=False)
#m_scan=m_scan_4.sample(frac=0.15,replace=False)
#m_syn=m_syn_4.sample(frac=0.25,replace=False)
#m_udp=m_udp_4.sample(frac=0.1,replace=False)
#m_p_udp=m_p_udp_4.sample(frac=0.27,replace=False)


benign['type']='benign'
g_combo['type']='gafgyt_combo'
#g_junk['type']='gafgyt_junk'
#g_scan['type']='gafgyt_scan'
#g_tcp['type']='gafgyt_tcp'
#g_udp['type']='gafgyt_udp'
#m_udp['type']='mirai_udp'
#m_ack['type']='mirai_ack'
#m_scan['type']='mirai_scan'
#m_syn['type']='mirai_syn'
#m_p_udp['type']='mirai_udpplain'

In [21]:
#data=pd.concat([benign,m_udp,g_combo,g_junk,g_scan,g_tcp,g_udp,m_ack,m_scan,m_syn,m_p_udp],
#               axis=0, sort=False, ignore_index=True)

In [22]:
data=pd.concat([benign,g_combo],
               axis=0, sort=False, ignore_index=True)

In [23]:
#how many instances of each class
data.groupby('type')['type'].count()

type
benign          43810
gafgyt_combo    14538
Name: type, dtype: int64

In [24]:
data.shape

(58348, 116)

In [25]:
#shuffle rows of dataframe 
sampler=np.random.permutation(len(data))
data=data.take(sampler)
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
7867,1.001985,66.000000,0.000000,1.024288,66.000000,2.680000e-07,1.407756,66.025275,1.561209,10.440252,...,-4.911862e+00,-9.399260e-02,18.140728,69.981442,6.917435,100.201273,75.789827,4.684111,0.088326,benign
30748,5.101295,66.287363,4.515233,5.811963,66.656133,1.006762e+01,7.259062,67.368859,20.197828,15.610674,...,-7.570000e-17,-6.210000e-12,20.107612,66.009505,1.312859,115.660788,342.725125,-0.037061,-0.001525,benign
22800,1.016381,81.742127,4.059469,1.085471,80.740148,1.857041e+01,1.537880,76.445103,60.835862,11.315241,...,-3.852861e+00,-9.552250e-02,16.770200,70.480777,7.184363,99.811421,73.927225,2.195677,0.042009,benign
1744,1.000010,66.000000,0.000000,1.001322,66.000000,9.090000e-13,1.183615,66.000000,0.000008,8.518496,...,1.861211e+00,5.859316e-02,17.015718,69.747145,6.775930,99.498981,71.461837,2.677028,0.053389,benign
17730,1.099307,65.779493,1.274420,1.387290,65.369307,3.386386e+00,2.631590,64.800374,5.768749,10.789430,...,8.000042e+01,1.262728e-01,19.502732,90.547869,40.074483,122.131886,1709.505917,77.304456,0.079693,benign


In [26]:
#dummy encode labels, store separately
labels_full=pd.get_dummies(data['type'], prefix='type')
labels_full.head()

,type_benign,type_gafgyt_combo
7867,1,0
30748,1,0
22800,1,0
1744,1,0
17730,1,0


In [27]:
#drop labels from training dataset
data=data.drop(columns='type')
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
7867,1.001985,66.000000,0.000000,1.024288,66.000000,2.680000e-07,1.407756,66.025275,1.561209,10.440252,...,76.887237,-4.911862e+00,-9.399260e-02,18.140728,69.981442,6.917435,100.201273,75.789827,4.684111,0.088326
30748,5.101295,66.287363,4.515233,5.811963,66.656133,1.006762e+01,7.259062,67.368859,20.197828,15.610674,...,163.619417,-7.570000e-17,-6.210000e-12,20.107612,66.009505,1.312859,115.660788,342.725125,-0.037061,-0.001525
22800,1.016381,81.742127,4.059469,1.085471,80.740148,1.857041e+01,1.537880,76.445103,60.835862,11.315241,...,68.081100,-3.852861e+00,-9.552250e-02,16.770200,70.480777,7.184363,99.811421,73.927225,2.195677,0.042009
1744,1.000010,66.000000,0.000000,1.001322,66.000000,9.090000e-13,1.183615,66.000000,0.000008,8.518496,...,50.417019,1.861211e+00,5.859316e-02,17.015718,69.747145,6.775930,99.498981,71.461837,2.677028,0.053389
17730,1.099307,65.779493,1.274420,1.387290,65.369307,3.386386e+00,2.631590,64.800374,5.768749,10.789430,...,915.731719,8.000042e+01,1.262728e-01,19.502732,90.547869,40.074483,122.131886,1709.505917,77.304456,0.079693


In [28]:
#standardize numerical columns
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/df[col].std()

data_st=data.copy()
for i in (data_st.iloc[:,:-1].columns):
    standardize (data_st,i)

data_st.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
7867,-0.582240,-0.170392,-0.094056,-0.580753,-0.173157,-0.096677,-0.575619,-0.175347,-0.105918,-0.567160,...,-0.144931,-0.040886,-0.693181,0.114257,-0.113468,-0.146584,-0.208837,-0.199090,-0.050936,0.088326
30748,-0.523876,-0.167392,-0.093911,-0.539133,-0.166037,-0.096358,-0.558324,-0.159704,-0.105315,-0.565456,...,-0.143120,-0.039829,-0.025059,0.194536,-0.162059,-0.278675,-0.080756,-0.193975,-0.051827,-0.001525
22800,-0.582035,-0.006050,-0.093925,-0.580221,-0.013211,-0.096088,-0.575234,-0.054032,-0.104001,-0.566872,...,-0.145115,-0.040658,-0.704056,0.058319,-0.107359,-0.140293,-0.212067,-0.199126,-0.051406,0.042009
1744,-0.582268,-0.170392,-0.094056,-0.580952,-0.173157,-0.096677,-0.576281,-0.175641,-0.105969,-0.567793,...,-0.145484,-0.039429,0.391435,0.068340,-0.116334,-0.149919,-0.214656,-0.199173,-0.051315,0.053389
17730,-0.580854,-0.172694,-0.094015,-0.577597,-0.180001,-0.096570,-0.572002,-0.189608,-0.105782,-0.567045,...,-0.127408,-0.022622,0.872518,0.169847,0.138137,0.634875,-0.027143,-0.167785,-0.037227,0.079693


# Different models

## DBSCAN - Density-Based Spatial Clustering of Applications with Noise

In [31]:
# Compute DBSCAN
db = DBSCAN(eps=4, min_samples=13000).fit(data_st) # variar eps e min_samples para encontrar n clusters desejado 
db.labels_

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [45]:
#db.labels_[0:100]

array([ 0,  1,  2,  0,  3,  4,  2,  4,  3,  0,  3,  3,  3,  3,  1,  3,  1,
        1,  3,  4,  1, -1, -1, -1,  3, -1,  4,  1,  3,  3, -1,  2,  2,  2,
        3,  4,  2, -1,  1,  4,  3,  4,  0,  4,  2,  4,  4,  3,  3,  0,  2,
        1,  3,  0, -1,  3,  3,  1,  2,  0,  1,  3,  1,  3,  4, -1,  1,  3,
        4,  3,  3,  4,  3,  1,  3,  4,  4,  0,  4, -1,  0,  2,  4,  3,  4,
        2,  4,  4, -1, -1,  3,  3,  0,  3,  4,  2,  1, -1, -1,  3],
      dtype=int64)

In [39]:
#labels_full[0:15]

,type_benign,type_gafgyt_combo,type_gafgyt_junk,type_gafgyt_scan,type_gafgyt_tcp,type_gafgyt_udp,type_mirai_ack,type_mirai_scan,type_mirai_syn,type_mirai_udp,type_mirai_udpplain
841,1,0,0,0,0,0,0,0,0,0,0
21811,0,0,0,0,0,0,0,0,0,0,1
6465,0,0,0,0,0,0,0,0,0,1,0
139,1,0,0,0,0,0,0,0,0,0,0
10252,0,0,0,1,0,0,0,0,0,0,0
17841,0,0,0,0,0,0,0,0,1,0,0
15475,0,0,0,0,0,0,1,0,0,0,0
7675,0,1,0,0,0,0,0,0,0,0,0
20498,0,0,0,0,0,0,0,0,1,0,0
446,1,0,0,0,0,0,0,0,0,0,0


In [17]:
# implementar metodo de comparação dos clusters com os labels para saber acerto

for x in range(10, 0, -1):
    eps = 1/(11-x)
    samples = 130*x
    db = DBSCAN(eps=eps, min_samples=samples).fit(data_st)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    
    print('EPS: ', eps, 'Min samples: ', samples)
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    


In [35]:
classe1 = 0
classe2 = 0
outlier = 0
correct = 0
uncorrect = 0
outlier2 = 0
for i in range(len(data_st)):
    if(db.labels_[i] == 0):
        classe1+=1
    elif(db.labels_[i] == 1):
        classe2+=1
    elif(db.labels_[i] == -1):
        outlier+=1
    
    if((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 1) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 1)):
        correct+=1
    elif((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 0) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 0)):
        uncorrect+=1
    else:
        outlier2+=1
    
    
    
    #dbscan_label = db.labels_[i]
print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', classe1, ' ', 'Malicious: ', classe2, ' ', 'Outlier: ', outlier)
print('Correct: ', correct,' ', 'Uncorrect: ', uncorrect,' ', 'Outliers: ', outlier2)

print('Accuracy: (%)', 'Benign data: ', classe1/len(benign), 'Malicious data: ', classe2/len(g_combo))
print('Complete data: ', correct/len(data))

Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  40271   Malicious:  14387   Outlier:  3690
Correct:  54651   Uncorrect:  7   Outliers:  3690
Accuracy: (%) Benign data:  0.9192193563113444 Malicious data:  0.9896134268812766
Complete data:  0.9366387879618838


## Isolation Forests (IF)

In [24]:
#from sklearn.ensemble import IsolationForest
#clf = IsolationForest(random_state=0).fit(data_st)
#prediction = clf.predict(data_st) # -1 => outlier

array([1, 1, 1, ..., 1, 1, 1])

## Local Outlier Factor (LOF)

## Elliptic Envelope (EE)

## One-Class Support Vector Machine (1CSVM)

## All 5 algorithms

In [ ]:
# Define "classifiers" to be used
algorithms = {
    "DBSCAN - Density-Based Spatial Clustering of Applications with Noise": DBSCAN(eps=4, min_samples=1300),
    "Isolation Forest": IsolationForest(random_state=0), 
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, contamination=0.1),
    "EllipticEnvelope": EllipticEnvelope(support_fraction=1., contamination=0.25),
    "OCSVM": OneClassSVM(nu=0.25, gamma=0.35)}

In [ ]:
for i, (alg_name, alg) in enumerate(algorithms.items()):
    alg.fit(data_st)
    ## show some results and evaluate them
    